In [75]:
%cd indice_invertido

[Errno 2] No such file or directory: 'indice_invertido'
/media/notebooks/indice_invertido


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [76]:
%%writefile mapper_indice.py
#!/usr/bin/env python3

import sys
import os
filename = os.environ.get("map_input_file","desconocido")

for line in sys.stdin:
    line = line.strip().split()
    for word in line:
        print(f"{word}\t{filename}")

Overwriting mapper_indice.py


In [89]:
%%writefile reducer_indice.py
#! /usr/bin/env python3

import sys

current_word = None
docs = set()

for line in sys.stdin:
    word,doc = line.strip().split("\t",1)

    if current_word == word:
        docs.add(doc)
    else:
        if current_word:
            print(f"{current_word}\t{','.join(sorted(docs))}")
        current_word = word
        docs = {doc}
if current_word:
            print(f"{current_word}\t{','.join(sorted(docs))}")

Overwriting reducer_indice.py


In [78]:
%%writefile doc1.txt
el cielo es azul

Overwriting doc1.txt


In [79]:
%%writefile doc2.txt
el sol es amarillo

Overwriting doc2.txt


In [80]:
%%writefile doc3.txt
el cielo es grande

Overwriting doc3.txt


In [81]:
!hdfs dfs -ls /

Found 6 items
drwxr-xr-x   - root supergroup          0 2025-11-13 10:07 /contar
drwxr-xr-x   - root supergroup          0 2025-11-13 09:22 /contar_palabras
drwxr-xr-x   - root supergroup          0 2025-11-18 09:05 /indice_invertido
drwxr-xr-x   - root supergroup          0 2025-11-18 09:12 /salida
drwxrwx---   - root supergroup          0 2025-11-12 11:32 /tmp
drwxrwxrwt   - root root                0 2025-11-18 09:12 /yarn


In [82]:
!hdfs dfs -mkdir /indice_invertido
!hdfs dfs -put doc*.txt /indice_invertido

mkdir: `/indice_invertido': File exists
put: `/indice_invertido/doc1.txt': File exists
put: `/indice_invertido/doc2.txt': File exists
put: `/indice_invertido/doc3.txt': File exists


In [ ]:
!hdfs dfs -ls /indice_invertido

Found 3 items
-rw-r--r--   3 root supergroup         17 2025-11-18 09:05 /indice_invertido/doc1.txt
-rw-r--r--   3 root supergroup         19 2025-11-18 09:05 /indice_invertido/doc2.txt
-rw-r--r--   3 root supergroup         19 2025-11-18 09:05 /indice_invertido/doc3.txt
Deleted /salida/indice_invertido


In [93]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper_indice.py -file reducer_indice.py -mapper mapper_indice.py -reducer reducer_indice.py -input /indice_invertido/ -output  /salida/indice_invertido

2025-11-18 09:20:03,721 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper_indice.py, reducer_indice.py, /tmp/hadoop-unjar7483544810070164739/] [] /tmp/streamjob6763344812973096464.jar tmpDir=null
2025-11-18 09:20:04,303 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.8:8032
2025-11-18 09:20:04,447 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.8:8032
2025-11-18 09:20:04,632 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763455228086_0002
2025-11-18 09:20:05,013 INFO mapred.FileInputFormat: Total input files to process : 3
2025-11-18 09:20:05,109 INFO mapreduce.JobSubmitter: number of splits:3
2025-11-18 09:20:05,215 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1763455228086_0002
2025-11-18 09:20:05,215 INFO mapreduce.JobSubmitter

In [94]:
!hdfs dfs -ls /salida/indice_invertido/
!hdfs dfs -cat /salida/indice_invertido/part-00000

Found 2 items
-rw-r--r--   3 root supergroup          0 2025-11-18 09:20 /salida/indice_invertido/_SUCCESS
-rw-r--r--   3 root supergroup        601 2025-11-18 09:20 /salida/indice_invertido/part-00000
amarillo	hdfs://namenode:9000/indice_invertido/doc2.txt
azul	hdfs://namenode:9000/indice_invertido/doc1.txt
cielo	hdfs://namenode:9000/indice_invertido/doc1.txt,hdfs://namenode:9000/indice_invertido/doc3.txt
el	hdfs://namenode:9000/indice_invertido/doc1.txt,hdfs://namenode:9000/indice_invertido/doc2.txt,hdfs://namenode:9000/indice_invertido/doc3.txt
es	hdfs://namenode:9000/indice_invertido/doc1.txt,hdfs://namenode:9000/indice_invertido/doc2.txt,hdfs://namenode:9000/indice_invertido/doc3.txt
grande	hdfs://namenode:9000/indice_invertido/doc3.txt
sol	hdfs://namenode:9000/indice_invertido/doc2.txt


In [90]:
cat doc1.txt | python3 mapper_indice.py | sort | python3 reducer_indice.py

azul	desconocido
cielo	desconocido
el	desconocido
es	desconocido
